block 1 to 6 is api integration

In [29]:
import pandas as pd
import tkinter as tk
import tkinter as tk
from tkinter import ttk
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import datetime     
import secret



In [5]:
api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)

In [6]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=x5ugrx91yexerba1&v=3


In [7]:
data = kite.generate_session("esUAEceVL8bIo5rh8GfX450KeIIW3F22", api_secret=api_secret)
kite.set_access_token(data["access_token"])
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)
print(data["access_token"])

qelT5XClsRsNLCSyZYi6en3DpkilJrrH


CALCULATE ALL THE LEVELS OF THE STOCKS (SWING HIGH AND SWING LOW)

In [8]:
def swingTop_bottom(monthly_data):
    data=[]
    length = len(monthly_data)

# n = int(inpucollectiont())  this will be used for specifying the counter for swing top
    for i in range(2,length-2):
        if ((monthly_data['high'][i] > monthly_data['high'][i+1]) and (monthly_data['high'][i] > monthly_data['high'][i+2]) and (monthly_data['high'] [i]> monthly_data['high'][i-1]) and (monthly_data['high'][i] > monthly_data['high'][i-2])):
            data.append(float(monthly_data['high'][i]))
         
        if ((monthly_data['low'][i] < monthly_data['low'][i+1]) and (monthly_data['low'][i] < monthly_data['low'][i+2]) and (monthly_data['low'][i] < monthly_data['low'][i-1]) and (monthly_data['low'][i] < monthly_data['low'][i-2])):
            data.append(float(monthly_data['low'][i]))
    

    return data

FUNCTION FOR CALCULATING THE NUMBER OF TAPS

In [9]:
def filterdTop(swingList,filteredata,levelAllowance):
    swingLen = len(swingList)
    filterdataLen = len(filteredata)
    # check=[]
    data ={}
    for i in range(swingLen):
        price =swingList[i]
        # print(price)
        percentage = ((price/100)*levelAllowance)
        subPercentage = price - percentage
        addPercentage = price + percentage
        # print(subPercentage,addPercentage)
        key = price
        # check.append('END')
        if key not in data:
            data[key]=0
        for j in range(filterdataLen):
            if ((float(filteredata['open'][j]) <= addPercentage) and (float(filteredata['open'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'open',float(filteredata['open'][j])])
            elif ((float(filteredata['high'][j]) <= addPercentage) and (float(filteredata['high'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'high',float(filteredata['high'][j])])
            elif ((float(filteredata['low'][j]) <= addPercentage) and (float(filteredata['low'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'low',float(filteredata['low'][j])])
            elif ((float(filteredata['close'][j]) <= addPercentage) and (float(filteredata['close'][j]) >= subPercentage)):
                data[key]+=1
                # check.append([(filteredata['date'][j]),'close',float(filteredata['close'][j])])
    
    return data

levels percentage

In [10]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge
       

function for first short listing of the stocks on the basis of 0.5%

In [11]:


def first_shortlist(symbol,addpercentage,subpercentage,ohlcCompare,close_data):
    stock_dict ={}
    stock_dict[symbol]=(set(),close_data)
    st = list(ohlcCompare.keys())
    close_add_percentage, close_sub_percentage = percentage(close_data, 4)
    for levels in range(len(st)):
       
        if (subpercentage <= float(st[levels]) <= addpercentage):
            
            if levels + 2 < len(st) and levels - 2 >= 0:
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels -1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels -2]]))
                
            elif levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                 

            elif levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]])) 

            elif levels - 1 >= 0 and levels + 2 < len(st): 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 2 < len(st) and close_sub_percentage <= float(st[levels + 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 2],ohlcCompare[st[levels + 2]])) 
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                 
                

            elif levels + 1 < len(st) and levels - 2 >= 0: 
                stock_dict[symbol][0].add((st[levels],ohlcCompare[st[levels]]))
                if levels + 1 < len(st) and close_sub_percentage <= float(st[levels + 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels + 1],ohlcCompare[st[levels + 1]])) 
                if levels - 1 >= 0 and close_sub_percentage <= float(st[levels - 1]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 1],ohlcCompare[st[levels - 1]])) 
                if levels - 2 >= 0 and close_sub_percentage <= float(st[levels - 2]) <= close_add_percentage: 
                    stock_dict[symbol][0].add((st[levels - 2],ohlcCompare[st[levels - 2]]))
                

   
    if stock_dict[symbol][0]:
        return stock_dict
    
    

this function take out the levels and there taps and this function will run at the end of the week

In [20]:
import json
watchlist ="niftyIndex"
timeframe = 'D'
instruments = kite.instruments("NSE")
nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
errorStockList=[]
levelTapping={}

In [15]:
datetime.datetime.now()

datetime.datetime(2024, 12, 13, 12, 51, 20, 153691)

In [21]:


for symbols in nifty50['Symbol']:
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)
        to_date = datetime.date.today()
        from_date = "2019-10-01"
        data = kite.historical_data(instrument_token, from_date, to_date, "day")
        df = pd.DataFrame(data)
        # print(df)
        if (timeframe == 'D'):
            df['date'] = pd.to_datetime(df['date']).dt.date  #.dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
            monthly_data = df.groupby('date').agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).reset_index()
        else:
            df['date'] = pd.to_datetime(df['date']).dt.to_period(timeframe) # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
            monthly_data = df.groupby('date').agg({
                'open': 'first',
                'high': 'max',
                'low': 'min',
                'close': 'last',
                'volume': 'sum'
            }).reset_index()
            
        st = swingTop_bottom(monthly_data) # important levels for each stock
        ohlcCompare = filterdTop(st,df,levelAllowance=0.15)
        levelTapping[symbols]=ohlcCompare
        # print(levelTapping)
        
    except StopIteration:
        errorStockList.append(symbols)
        continue
print(levelTapping)
jasonfile = {k: json.dumps(v) for k,v in levelTapping.items()}
df= pd.DataFrame(list(jasonfile.items()),columns=["stocks","levels"])
df.to_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\{watchlist}.csv",index=False)

{'NIFTY SERV SECTOR': {15599.75: 4, 16287.8: 8, 15890.8: 15, 16504.2: 4, 16915.35: 16, 16601.15: 8, 16931.05: 15, 16749.85: 9, 17175.4: 11, 16912.0: 14, 17179.1: 13, 16802.35: 15, 17609.85: 17, 17400.45: 17, 17656.55: 10, 17429.75: 19, 17626.9: 13, 17081.55: 8, 17743.2: 5, 17539.5: 18, 17721.2: 9, 17243.35: 11, 17533.35: 22, 16698.2: 9, 17567.7: 19, 17362.25: 11, 17724.8: 8, 17201.25: 15, 17578.55: 18, 16569.5: 5, 12363.55: 10, 10401.3: 1, 12902.1: 3, 10913.6: 1, 12393.75: 8, 11692.8: 2, 12719.1: 2, 11910.4: 5, 12579.7: 10, 13501.45: 7, 12248.45: 5, 12003.65: 8, 12873.8: 2, 11618.55: 3, 12166.3: 5, 11697.75: 4, 13589.5: 5, 13742.75: 4, 12636.05: 5, 14064.15: 4, 13380.45: 4, 14526.7: 11, 13868.0: 3, 14992.8: 12, 14513.15: 13, 14862.8: 11, 14184.8: 3, 14783.9: 18, 14360.5: 4, 14813.4: 7, 15585.5: 4, 14862.1: 11, 15251.9: 3, 14556.65: 14, 15023.05: 5, 13884.95: 4, 16061.05: 11, 16059.9: 11, 15466.0: 4, 16231.35: 7, 16173.45: 10, 15537.1: 2, 17879.5: 4, 17488.7: 20, 18165.2: 5, 17791.45: 4

This function will sort the stocks on the basis of intraday(0.5) or positional(1) and on the basis of taps in descending order

In [22]:
from datetime import date
import ast
import schedule
import time,datetime

def taps():
    print(datetime.datetime.now().time())
    today = date.today()
    now=  datetime.datetime.now()
    instruments = kite.instruments("NSE")
    levelAllowance = 0.5
    nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
    leveltap = pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\{watchlist}.csv")
    leveltap['levels'] = leveltap['levels'].apply(json.loads)
    i=0
    first_sort= {}
    levelTapping ={}
    errorStockList=[]
    for symbols in nifty50['Symbol']:
        try:
            instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)      
            try:
                # data = kite.historical_data(instrument_token,"2024-11-21","2024-11-21", "day")
                # close_data = data[0]['close']
                ltp = kite.ltp('NSE:'+symbols)
                close_data = ltp['NSE:'+symbols]['last_price']
                # time.sleep(1)
            except:
                print('error')
            addpercentage,subpercentage = percentage(close_data,levelAllowance) #THIS VALUES REPRESENTS THE RANGE FROM LTP
            sortedlist = first_shortlist(symbols,addpercentage,subpercentage,leveltap.iloc[i]['levels'],close_data)
            i+=1
            if sortedlist != None:
                first_sort[instrument_token] = sortedlist
        except StopIteration:
            errorStockList.append(symbols)
            continue
    
    flattened_data = []
    for key,values in first_sort.items():
        for symbol,prices in values.items():
            price_set =  list(prices[0])
            flattened_data.append([key,symbol,price_set,prices[1]])
    first_sorted_dataframe =  pd.DataFrame(flattened_data,columns=['token','symbol','levels','ltp'])
    first_sorted_csv = first_sorted_dataframe.to_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv",index=False)
    csvFile= pd.read_csv('E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv')

    csvFile['levels'] = csvFile['levels'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
    df = csvFile.explode('levels')
    df[['level','taps']] = pd.DataFrame(df['levels'].tolist(), index = df.index)
    df=df.drop(columns='levels')
    df = df.sort_values(by='taps', ascending=False)
    if levelAllowance==1:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_positional_{today.strftime("%d_%m_%y")}.csv',index=False)
    else:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_intraday_{today.strftime("%d_%m_%y")}_{now.strftime("%H_%M_%S")}.csv',index=False)
        
    print(errorStockList)
    print(len(errorStockList))

In [23]:
def taps2():
    print(datetime.datetime.now().time())
    today = date.today()
    now=  datetime.datetime.now()
    instruments = kite.instruments("NSE")
    levelAllowance = 1
    nifty50 =  pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\watchList\\{watchlist}.csv")
    leveltap = pd.read_csv(f"E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\{watchlist}.csv")
    leveltap['levels'] = leveltap['levels'].apply(json.loads)
    i=0
    first_sort= {}
    levelTapping ={}
    errorStockList=[]
    for symbols in nifty50['Symbol']:
        try:
            instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbols)      
            try:
                # ohlc = kite.ohlc('NSE:'+symbols)
                # close_data =  ohlc['NSE:'+symbols]['last_price']
                # print(ohlc)
                # data = kite.historical_data(instrument_token,"2024-12-11","2024-12-11", "day")
                # close_data = data[0]['close']
                ltp = kite.ltp('NSE:'+symbols)
                close_data = ltp['NSE:'+symbols]['last_price']
                print(close_data)
                # time.sleep(2)
            except:
                print('error')
            addpercentage,subpercentage = percentage(close_data,levelAllowance) #THIS VALUES REPRESENTS THE RANGE FROM LTP
            sortedlist = first_shortlist(symbols,addpercentage,subpercentage,leveltap.iloc[i]['levels'],close_data)
            i+=1
            if sortedlist != None:
                first_sort[instrument_token] = sortedlist
        except StopIteration:
            errorStockList.append(symbols)
            continue
    
    flattened_data = []
    for key,values in first_sort.items():
        for symbol,prices in values.items():
            price_set =  list(prices[0])
            flattened_data.append([key,symbol,price_set,prices[1]])
    first_sorted_dataframe =  pd.DataFrame(flattened_data,columns=['token','symbol','levels','ltp'])
    first_sorted_csv = first_sorted_dataframe.to_csv("E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv",index=False)
    csvFile= pd.read_csv('E:\\TRADING KARNAL\\OHLCcompare\\otherImp\\first_sorted_list_csv.csv')
    csvFile['levels'] = csvFile['levels'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
    df = csvFile.explode('levels')
    df[['level','taps']] = pd.DataFrame(df['levels'].tolist(), index = df.index)
    df=df.drop(columns='levels')
    df = df.sort_values(by='taps', ascending=False)
    if levelAllowance==1:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_positional_{today.strftime("%d_%m_%y")}.csv',index=False)
    else:
        df.to_csv(f'E:\\TRADING KARNAL\\OHLCcompare\\tapList\\tapList{watchlist}_{timeframe}_intraday_{today.strftime("%d_%m_%y")}_{now.strftime("%H_%M_%S")}.csv',index=False)
        
    print(errorStockList)
    print(len(errorStockList))

In [ ]:
# taps()
taps2()

In [ ]:

def submit():
    print(f"Watchlist: {watchlist_var.get()}")
    print(f"Timeframe: {timeframe_var.get()}")
    print(f"Range: {range_var.get()}")

# Create the main window
root = tk.Tk()
root.title("Dropdown Selection")

# Create variables for storing the selected values
watchlist_var = tk.StringVar()
timeframe_var = tk.StringVar()
range_var = tk.StringVar()

# Create the dropdowns
watchlist_label = ttk.Label(root, text="Watchlist")
watchlist_label.pack(pady=5)
watchlist_dropdown = ttk.Combobox(root, textvariable=watchlist_var, values=["FNO's", "nifty50", "nifty500","niftyIndices"])
watchlist_dropdown.pack(pady=5)

timeframe_label = ttk.Label(root, text="Timeframe")
timeframe_label.pack(pady=5)
timeframe_dropdown = ttk.Combobox(root, textvariable=timeframe_var, values=["W", "M"])
timeframe_dropdown.pack(pady=5)

range_label = ttk.Label(root, text="Range")
range_label.pack(pady=5)
range_dropdown = ttk.Combobox(root, textvariable=range_var, values=["positional", "intraday"])
range_dropdown.pack(pady=5)

# Create the submit button
submit_button = ttk.Button(root, text="Submit", command=submit)
submit_button.pack(pady=20)

# Start the Tkinter event loop
root.mainloop()



Watchlist: Watchlist2
Timeframe: 15min
Range: 1 month
Watchlist: Watchlist3
Timeframe: 1min
Range: 1 week


In [13]:
def everyHour():
    schedule.every().hour.do(taps)

    while True:
        schedule.run_pending()
        time.sleep(1)

def atTheEnd():
    schedule.every().day.at("13:00").do(taps2)

    while True:
        schedule.run_pending()
        time.sleep(1)

In [13]:
def check():
    print('hello')


def check2():
    print('world')

In [14]:
import threading


t1 =  threading.Thread(target=check)
t2 = threading.Thread(target=check2)

t1.start()
t2.start()

t1.join()
t2.join()

hello
world


In [ ]:
import smtplib
smtp_server= smtplib.SMTP('smtp.gmail.com',587)
smtp_server.ehlo()
smtp_server.starttls()
smtp_server.login('sudipto11bhukta@gmail.com',"")
smtp_server.sendmail("sudipto11bhukta@gmail.com","tradernaman1988@gmail.com","hello from sudipto this is a test mail")
smtp_server.quit()

(220, b'2.0.0 Ready to start TLS')